In [67]:
from hossam import *
origin = load_data("restaurant_sales")

어느 식당의 1년간 일별 매출을 기록한 데이터로, 매출액을 종속변수로 하고 방문객 수, 객단가, 광고비, 날씨, 요일, 식당 유형 등의 정보를 포함함.

컬럼명           변수 타입    데이터 성격    설명
---------------  -----------  -------------  -------------------------------------------------------
date             날짜         시계열 인덱스  관측 날짜 (1년, 일 단위)
sales            연속형       수치           일 매출액 (종속변수). 이상치·왜도 존재 → 로그 변환 대상
visitors         연속형       수치           일 방문객 수. 매출과 강한 양의 관계
avg_price        연속형       수치           평균 객단가. 양의 왜도 → 로그 변환 권장
marketing_cost   연속형       수치           일 광고비 지출. 이상치·결측치 포함
delivery_ratio   연속형       비율           배달 매출 비중 (0~1). 비선형 효과 가능
rain_mm          연속형       수치           일 강수량(mm). 일부 결측치 존재
temperature      연속형       수치           일 평균 기온(℃). 계절성 반영
holiday          명목형       이진           공휴일 여부 (0=평일, 1=공휴일)
weekend          명목형       이진           주말 여부 (0=평일, 1=주말)
restaurant_type  명목형       범주           식당 유형 (한식/중식/양식/카페) → 원핫인코딩 필요



In [68]:
from hossam import *

#파이썬 기본 라이브러리 
from pandas import DataFrame, concat, get_dummies
import numpy as np
from itertools import combinations

#시각화 라이브러리
import seaborn as sb
from matplotlib import pyplot as plt

#통계 라이브러리
import  statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import linear_reset, het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova
from scipy.stats import (
    normaltest,
    bartlett, 
    ttest_ind,
    spearmanr,
    shapiro,
    jarque_bera,
    t,
    f,
)

#머신러닝 라이브러리
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve

from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    SGDRegressor,
)

from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)



In [69]:
origin = load_data("restaurant_sales")

어느 식당의 1년간 일별 매출을 기록한 데이터로, 매출액을 종속변수로 하고 방문객 수, 객단가, 광고비, 날씨, 요일, 식당 유형 등의 정보를 포함함.

컬럼명           변수 타입    데이터 성격    설명
---------------  -----------  -------------  -------------------------------------------------------
date             날짜         시계열 인덱스  관측 날짜 (1년, 일 단위)
sales            연속형       수치           일 매출액 (종속변수). 이상치·왜도 존재 → 로그 변환 대상
visitors         연속형       수치           일 방문객 수. 매출과 강한 양의 관계
avg_price        연속형       수치           평균 객단가. 양의 왜도 → 로그 변환 권장
marketing_cost   연속형       수치           일 광고비 지출. 이상치·결측치 포함
delivery_ratio   연속형       비율           배달 매출 비중 (0~1). 비선형 효과 가능
rain_mm          연속형       수치           일 강수량(mm). 일부 결측치 존재
temperature      연속형       수치           일 평균 기온(℃). 계절성 반영
holiday          명목형       이진           공휴일 여부 (0=평일, 1=공휴일)
weekend          명목형       이진           주말 여부 (0=평일, 1=주말)
restaurant_type  명목형       범주           식당 유형 (한식/중식/양식/카페) → 원핫인코딩 필요



In [70]:
df1 = origin.astype(
    {"restaurant_type": "category", "holiday":"category","weekend":"category" }
)
df1.set_index("date", inplace=True)
display(df1.info())
df1.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2024-01-01 to 2024-12-30
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   sales            365 non-null    int64   
 1   visitors         365 non-null    int64   
 2   avg_price        365 non-null    int64   
 3   marketing_cost   359 non-null    float64 
 4   delivery_ratio   365 non-null    float64 
 5   rain_mm          359 non-null    float64 
 6   temperature      365 non-null    float64 
 7   holiday          365 non-null    category
 8   weekend          365 non-null    category
 9   restaurant_type  365 non-null    category
dtypes: category(3), float64(4), int64(3)
memory usage: 24.3 KB


None

,sales,visitors,avg_price,marketing_cost,delivery_ratio,rain_mm,temperature,holiday,weekend,restaurant_type
date,,,,,,,,,,
2024-01-01,10525760,122,14490,9900.0,0.48,6.0,5.2,0,0,한식
2024-01-02,8931390,106,11880,9400.0,0.48,7.0,11.1,0,0,중식
2024-01-03,12654970,120,18010,5800.0,0.43,12.4,12.9,1,0,한식
2024-01-04,9650090,115,11160,6300.0,0.41,6.6,12.9,0,0,중식
2024-01-05,9951380,124,15480,5400.0,0.51,8.0,11.0,0,0,카페


### 결측치 확인

In [71]:
n = len(df1)

missing_df = DataFrame({"missing_count": df1.isnull().sum()})
missing_df["missing_ratio"] = missing_df["missing_count"]/n

display(missing_df)

,missing_count,missing_ratio
sales,0,0.000000
visitors,0,0.000000
avg_price,0,0.000000
marketing_cost,6,0.016438
delivery_ratio,0,0.000000
rain_mm,6,0.016438
temperature,0,0.000000
holiday,0,0.000000
weekend,0,0.000000
restaurant_type,0,0.000000


### 독립검정 t-test함수 정의

In [72]:
def hs_ttest_ind(x_data, y_data):
    #정규성 판별
    _,  p1 = normaltest(x_data)
    _, p2 = normaltest(y_data)
    normal_dist = p1>0.05 and p2> 0.05

    #등분산성 판별
    if normal_dist:
        method_name="Bartlett"
        s, p = levene(x_data, y_data)
    else:
        method_name = "Levene"
        s, p = levene(x_data, y_data)
    equal_var = p>0.05

    #독립표본 t-검정 수행
    alternative: list =["two-sided", "less", "greater"]
    result: list = []
    fmt: str = "u(x) {0} u(y)"

    for a in alternative:
        s, p = ttest_ind(x_data, y_data, equal_var=equal_var, alternative=a)
        n="t-test_ind" if equal_var else "Welch's t-test"

        #검정 결과 해석
        itp = None

        if a == "two-sided":
            itp = fmt.format("==" if p>0.05 else "!=") #type: ignore
        elif a=="less":
            itp = fmt.format(">=" if p>0.05 else "<") #type: ignore
        else:
            itp = fmt.format("<=" if p>0.05 else ">") #type: ignore

        result.append(
            {
                "test":n,
                "alternative":a,
                "interpretation": itp,
                "normaltest":normal_dist,
                method_name:equal_var,
                n: round(s, 3),
                "p-value":round(p, 4),
                "H0":p>0.05,
                "H1": p<=0.05,
            }
        )
    rdf = DataFrame(result)
    rdf.set_index(["test", "alternative"], inplace=True)
    return rdf

### marketing_cost 삭제 전후의 종속변수에 대해 평균적으로 유의미한 변화가 있는지 확인

In [73]:
df_temp = df1.dropna(subset=["marketing_cost"])
hs_ttest_ind(origin["sales"], df_temp["sales"])

interpretation  normaltest  Levene  t-test_ind  \
test       alternative                                                  
t-test_ind two-sided     u(x) == u(y)       False    True      -0.041   
           less          u(x) >= u(y)       False    True      -0.041   
           greater       u(x) <= u(y)       False    True      -0.041   

                        p-value    H0     H1  
test       alternative                        
t-test_ind two-sided     0.9673  True  False  
           less          0.4836  True  False  
           greater       0.5164  True  False

In [74]:
hs_stats.ttest_ind(x=origin["sales"], y=df_temp["sales"])

interpretation  equal_var_checked  statistic  p-value  \
test       alternative                                                         
t-test_ind two-sided     μ(x) == μ(y)               True     -0.041   0.9673   
           less          μ(x) >= μ(y)               True     -0.041   0.4836   
           greater       μ(x) <= μ(y)               True     -0.041   0.5164   

                          H0     H1  
test       alternative               
t-test_ind two-sided    True  False  
           less         True  False  
           greater      True  False

### rain_mm 삭제 전후의 종속변수에 대해 평균적으로 유의미한 변화가 있는지 확인

In [75]:
df_temp = df1.dropna(subset=["rain_mm"])
df1_target = df1["sales"]
df_temp_target = df_temp["sales"]

hs_ttest_ind(df1_target, df_temp_target)

interpretation  normaltest  Levene  t-test_ind  \
test       alternative                                                  
t-test_ind two-sided     u(x) == u(y)       False    True       0.147   
           less          u(x) >= u(y)       False    True       0.147   
           greater       u(x) <= u(y)       False    True       0.147   

                        p-value    H0     H1  
test       alternative                        
t-test_ind two-sided     0.8830  True  False  
           less          0.5585  True  False  
           greater       0.4415  True  False

### 모든 결측치 일괄 삭제 전후의 종속변수에 대해 평균적으로 유의미한 변화가 있는지 확인


In [76]:
df_temp = df1.dropna()
df1_target = df1["sales"]
df_temp_target = df_temp["sales"]

hs_ttest_ind(df1_target, df_temp_target)

interpretation  normaltest  Levene  t-test_ind  \
test       alternative                                                  
t-test_ind two-sided     u(x) == u(y)       False    True       0.107   
           less          u(x) >= u(y)       False    True       0.107   
           greater       u(x) <= u(y)       False    True       0.107   

                        p-value    H0     H1  
test       alternative                        
t-test_ind two-sided     0.9144  True  False  
           less          0.5428  True  False  
           greater       0.4572  True  False

In [77]:
df2 = df1.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 353 entries, 2024-01-01 to 2024-12-30
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   sales            353 non-null    int64   
 1   visitors         353 non-null    int64   
 2   avg_price        353 non-null    int64   
 3   marketing_cost   353 non-null    float64 
 4   delivery_ratio   353 non-null    float64 
 5   rain_mm          353 non-null    float64 
 6   temperature      353 non-null    float64 
 7   holiday          353 non-null    category
 8   weekend          353 non-null    category
 9   restaurant_type  353 non-null    category
dtypes: category(3), float64(4), int64(3)
memory usage: 23.5 KB


## 이상치 확인

In [78]:
def hs_outlier_table(data:DataFrame):
    fields = data.select_dtypes(include=np.number).columns

    result = []
    for f in fields:
        q1 = data[f].quantile(q=0.25)
        q2 = data[f].quantile(q=0.5)
        q3 = data[f].quantile(q=0.75)
        min_value = data[f].min()
        max_value = data[f].max()

        #이상치 경계 (Tukey's fences)
        iqr = q3-q1
        down = q1 - 1.5*iqr
        up = q3 + 1.5*iqr

        # 왜도
        skew = data[f].skew()

        #이상치 개수 및 비율
        outlier_count = ((data[f]<down)| (data[f]>up)).sum()
        outlier_rate = (outlier_count/ len(data)) *100

        # 왜도
        skew = data[f].skew()

        #분포 특성 판정(왜도 기준)
        abs_skew = abs(skew)
        if abs_skew<0.5:
            dist = "거의 대칭"
        elif abs_skew<1.0:
            if skew>0:
                dist="약한 우측 꼬리"
            else:
                dist = "약한 좌측 꼬리"
        elif abs_skew<2.0:
            if skew>0:
                dist="중간 우측 꼬리"
            else:
                dist="중간 좌측 꼬리"
        else: 
            if skew >0:
                dist = "극단 우측 꼬리"
            else:
                dist = "극단 좌측 꼬리"

        # 로그변환 필요성 판정
        if abs_skew<0.5:
            log_need = "낮음"
        elif abs_skew<1.0:
            log_need="중간"
        else:
            log_need = "높음"
        iq = {
            "field":f,
            "q1":q1,
            "q2":q2,
            "q3":q3,
            "iqr":iqr,
            "up":up,
            "down":down,
            "min":min_value,
            "max":max_value,
            "outlier_count":outlier_count,
            "outlier_rate": outlier_rate,
            "skew":skew,
            "dist":dist,
            "log_need":log_need,
        }
        result.append(iq)
    return DataFrame(result).set_index("field")

In [79]:
outlier=hs_outlier_table(df2)
outlier

,q1,q2,q3,iqr,up,down,min,max,outlier_count,outlier_rate,skew,dist,log_need
field,,,,,,,,,,,,,
sales,9620430.00,12203340.00,16828290.00,7207860.00,2.764008e+07,-1191360.000,5972790.00,38585880.00,10,2.832861,1.256981,중간 우측 꼬리,높음
visitors,113.00,120.00,126.00,13.00,1.455000e+02,93.500,83.00,151.00,6,1.699717,-0.014914,거의 대칭,낮음
avg_price,12480.00,15340.00,17550.00,5070.00,2.515500e+04,4875.000,6330.00,27280.00,5,1.416431,0.471286,거의 대칭,낮음
marketing_cost,4500.00,6800.00,10500.00,6000.00,1.950000e+04,-4500.000,1400.00,51700.00,12,3.399433,2.628738,극단 우측 꼬리,높음
delivery_ratio,0.36,0.45,0.53,0.17,7.850000e-01,0.105,0.17,0.77,0,0.000000,0.064028,거의 대칭,낮음
rain_mm,2.10,4.60,7.60,5.50,1.585000e+01,-6.150,0.00,20.00,8,2.266289,1.117849,중간 우측 꼬리,높음
temperature,-0.40,10.00,20.50,20.90,5.185000e+01,-31.750,-10.40,29.50,0,0.000000,0.011906,거의 대칭,낮음


# 명목형 변수 anova

In [80]:
def hs_oneway_anova(data: DataFrame, dv:str, between:str, alpha: float =0.05):
    #결측치 제거
    df_filtered = data[[dv, between]].dropna()

    #---------------------------------------------
    #정규성 검정
    #---------------------------------------------
    #그룹별로 이름 추출
    group_names = sorted(df_filtered[between].unique())

    #모든 그룹이 정규성을 충족하는지 여부
    normality_satisfied = True

    for group in group_names:
        group_values = df_filtered[df_filtered[between] == group][dv].dropna()
        if len(group_values)>0:
            s, p = normaltest(group_values)
            if p<=alpha:
                normality_satisfied = False
                break

    #-------------------------------------------------
    # 2. 등분산성 검정
    #-------------------------------------------------
    # 각 그룹별로 데이터 분리
    group_data_dict = {}
    for group in group_names:
        group_data_dict[group] = (
            df_filtered[df_filtered[between]==group][dv].dropna().values
        )
    #등분산 검정 수행
    if len(group_names)>1:
        if normality_satisfied:
            s, p = bartlett(*group_data_dict.values())
            en = "Bartlett"
        else:
            #정규성을 만족하지 않으면 Levene검정
            s, p = levene(*group_data_dict.values())
            en = "Levene"
        #등분산성 충족 여부
        equal_var_satisfied = p>alpha
    else:
        equal_var_satisfied =True

    #---------------------------------------------
    # 3.ANOVA 수행
    #----------------------------------------------
    anova_df:DataFrame
    anova_method:str

    if equal_var_satisfied:
        #등분산을 만족할 때 일반적인 ANOVA 사용
        anova_method = "ANOVA"
        anova_df = anova(data=df_filtered, dv=dv, between=between)

    else:
        #등분산을 만족하지 않을 때 Welch's ANOVA 사용
        anova_method="Welch"
        anova_df = anova(data=df_filtered, dv=dv, between=between)
    
    # ANOVA 결과에 메타정보 추가 
    anova_df.insert(1, "normality", normality_satisfied)
    anova_df.insert(2, en, equal_var_satisfied)
    anova_df[anova_method] = anova_df["p-unc"] <=alpha if "p-unc " in anova_df.columns else False
    return anova_df

In [81]:
hs_oneway_anova(df2, dv="sales", between = "restaurant_type")

,Source,normality,Levene,ddof1,ddof2,F,p-unc,np2,ANOVA
0,restaurant_type,False,True,3,349,0.208264,0.890657,0.001787,False


In [ ]:
anova_df = None

fields = df2.dtypes[df2.dtypes == "category"].index.tolist()

for f in fields:
    anova_result = hs_oneway_anova(df2, dv="sales", between=f)
    if anova_df is None: 
        anova_df = anova_result
    else:
        anova_df = concat([anova_df, anova_result], ignore_index=True)

anova_df

,Source,normality,Levene,ddof1,ddof2,F,p-unc,np2,Welch,ANOVA
0,holiday,False,False,1,351,370.406423,7.300924e-57,0.513450,False,NaN
1,weekend,False,False,1,351,446.423380,1.617975e-64,0.559832,False,NaN
2,restaurant_type,False,True,3,349,0.208264,8.906571e-01,0.001787,NaN,False
